In [ ]:
%matplotlib widget

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from qualang_tools.units import unit
u = unit(coerce_to_integer=True)
from qm.qua import *
from qualang_tools.loops import from_array
import QM
import progress
import time
import h5py

# Exercice 0: Période des oscillations de Rabi
Modifiez l’amplitude de l’impulsion à l’aide de la directive amp et observez comment la fréquence des oscillations de Rabi est modifiée.

In [ ]:
# Parameters Definition
n_avg = 500  # The number of averages
# Pulse duration sweep (in clock cycles = 4ns) - must be larger than 4 clock cycles
t_min = 4 // 4
t_max = 1000 // 4
dt = 8 // 4
durations = np.arange(t_min, t_max, dt)
thermalization_time = 80 #in µs

###################
# The QUA program #
###################
with program() as qmprog:
    n = declare(int)  # QUA variable for the averaging loop
    t = declare(int)  # QUA variable for the qubit pulse duration
    I = declare(fixed)  # QUA variable for the measured 'I' quadrature
    Q = declare(fixed)  # QUA variable for the measured 'Q' quadrature
    I_st = declare_stream()  # Stream for the 'I' quadrature
    Q_st = declare_stream()  # Stream for the 'Q' quadrature

    with for_(n, 0, n < n_avg, n + 1):  # QUA for_ loop for averaging
        with for_(*from_array(t, durations)):  # QUA for_ loop for sweeping the pulse duration
            play("trigger","trigger")
            # Play the qubit pulse with a variable duration (in clock cycles = 4ns)
            play("pi"*amp(1.0), "qubit", duration=t)
            # Align the two elements to measure after playing the qubit pulse.
            align("qubit", "resonator")
            # Measure the state of the resonator
            measure(
                "readout",
                "resonator",
                dual_demod.full("cos", "sin", I),
                dual_demod.full("minus_sin", "cos", Q),
            )
            # Wait for the qubit to decay to the ground state
            wait(thermalization_time * u.us, "resonator")
            # Save the 'I' & 'Q' quadratures to their respective streams
            save(I, I_st)
            save(Q, Q_st)

    with stream_processing():
        # Cast the data into a 1D vector, average the 1D vectors together and store the results on the OPX processor
        I_st.buffer(len(durations)).average().save("I")
        Q_st.buffer(len(durations)).average().save("Q")

# Send the QUA program to the OPX, which compiles and executes it
job = QM.Job(qmprog)

# Non interacting version
# job = QM.JobSimple(qmprog)
# Wait for the job to finish
# job.wait()

In [ ]:
# Create plot
I,Q=job.get_results("I", "Q")
fig,ax = plt.subplots()
ax.plot(4*durations, I*1e4, 4*durations, Q*1e4)
ax.set_ylabel("Quadrature")
ax.set_xlabel("Pulse Duration [ns]")
ax.legend(('I','Q'))

## Exercise 1: Ajustement de l'amplitude du pulse $\pi$  gaussien
L’objectif est de trouver l’amplitude de la micro-onde (MW) permettant de faire tourner le qubit de $\pi$ en utilisant la durée d’impulsion gaussienne prédéfinie.

- Jouer l’impulsion `x180` sur le qubit et modifier son amplitude dans une boucle à l’aide de la directive `amp`.
- Tracer les composantes I et Q du signal de lecture en fonction de l’amplitude.


In [ ]:
# Parameters Definition
n_avg = 500  # The number of averages
amplitudes = np.arange(0, 1.2, 0.01)
thermalization_time = 80 #in µs

###################
# The QUA program #
###################
with program() as qmprog:
    n = declare(int)  # QUA variable for the averaging loop
    amplitude = declare(fixed)  
    I = declare(fixed)  # QUA variable for the measured 'I' quadrature
    Q = declare(fixed)  # QUA variable for the measured 'Q' quadrature
    I_st = declare_stream()  # Stream for the 'I' quadrature
    Q_st = declare_stream()  # Stream for the 'Q' quadrature
    n_st = declare_stream()  # Stream for the averaging iteration 'n'

    with for_(n, 0, n < n_avg, n + 1):  # QUA for_ loop for averaging
        with for_(*from_array(amplitude, amplitudes)): 
            # Play the qubit pulse with a variable duration (in clock cycles = 4ns)
            play("x180"*amp(1.0), "qubit") # <--- Modify to change the amplitude with the loop variable
            # Align the two elements to measure after playing the qubit pulse.
            align("qubit", "resonator")
            # Measure the state of the resonator
            measure(
                "readout",
                "resonator",
                dual_demod.full("cos", "sin", I),
                dual_demod.full("minus_sin", "cos", Q),
            )
            # Wait for the qubit to decay to the ground state
            wait(thermalization_time * u.us, "resonator")
            # Save the 'I' & 'Q' quadratures to their respective streams
            save(I, I_st)
            save(Q, Q_st)

    with stream_processing():
        # Cast the data into a 1D vector, average the 1D vectors together and store the results on the OPX processor
        I_st.buffer(len(amplitudes)).average().save("I")
        Q_st.buffer(len(amplitudes)).average().save("Q")

# Send the QUA program to the OPX, which compiles and executes it
job = QM.Job(qmprog)

# Non interacting version
# job = QM.JobSimple(qmprog)
# Wait for the job to finish
# job.wait()

In [ ]:
# Create plot
I,Q=job.get_results("I", "Q")
fig,ax = plt.subplots()
ax.plot(amplitudes, I*1e4, amplitudes, Q*1e4)
ax.set_ylabel("Quadrature")
ax.legend(('I','Q'))
ax.set_xlabel("Pulse Amplitude Modifier")

## Exercice 2 : Déterminer le temps de vie de l’état excité du qubit ($T_1$)

- Écrire une séquence où le qubit est préparé dans l’état $|1\rangle$ au début à l’aide d’une impulsion `x180`.
- Retarder la mesure d’un temps variable, modifié dans une boucle.
- Tracer les composantes I et Q du signal de lecture en fonction du délai.
- Estimer le temps $T_1$ du qubit.


In [ ]:
# Parameters Definition
n_avg = 2000  # The number of averages
# Pulse duration sweep (in clock cycles = 4ns) - must be larger than 4 clock cycles
t_min = 4 // 4
t_max = 200000 // 4
dt = 10000 // 4
durations = np.arange(t_min, t_max, dt)
thermalization_time = 80 #in µs

###################
# The QUA program #
###################
with program() as qmprog:
    n = declare(int)  # QUA variable for the averaging loop
    t = declare(int)  # QUA variable for the qubit pulse duration
    I = declare(fixed)  # QUA variable for the measured 'I' quadrature
    Q = declare(fixed)  # QUA variable for the measured 'Q' quadrature
    I_st = declare_stream()  # Stream for the 'I' quadrature
    Q_st = declare_stream()  # Stream for the 'Q' quadrature

    with for_(n, 0, n < n_avg, n + 1):  # QUA for_ loop for averaging
        with for_(*from_array(t, durations)):  # QUA for_ loop for sweeping the waiting time
            # Write the sequence
            # ....
            # Measure the state of the resonator
            measure(
                "readout",
                "resonator",
                dual_demod.full("cos", "sin", I),
                dual_demod.full("minus_sin", "cos", Q),
            )
            # Wait for the qubit to decay to the ground state
            wait(thermalization_time * u.us, "resonator")
            # Save the 'I' & 'Q' quadratures to their respective streams
            save(I, I_st)
            save(Q, Q_st)

    with stream_processing():
        # Cast the data into a 1D vector, average the 1D vectors together and store the results on the OPX processor
        I_st.buffer(len(durations)).average().save("I")
        Q_st.buffer(len(durations)).average().save("Q")

# Send the QUA program to the OPX, which compiles and executes it
job = addjob(qmprog, qmm)

In [ ]:
# Create plot
I,Q=job.get_results("I", "Q")
fig,ax = plt.subplots()
ax.plot(4*durations*1e-3, I*1e4, 4*durations*1e-3, Q*1e4)
ax.set_ylabel("Quadrature")
ax.set_xlabel("Pulse Delay [µs]")
ax.legend(('I','Q'))

## Exercice 3 : Déterminer le temps de cohérence de phase du qubit ($T_2$)

- Lire la documentation sur la [séquence de Ramsey](https://qiskit-community.github.io/qiskit-experiments/manuals/characterization/t2ramsey.html).
- Écrire la séquence de Ramsey en QUA en utilisant deux impulsions `x90` séparées par un délai variable, modifié dans une boucle.
- Inclure également une commande `frame_rotation_2pi` qui fait tourner le repère d’une phase proportionnelle au délai (utiliser une addition ou une multiplication ; la lecture de la documentation sur le [casting](https://docs.quantum-machines.co/latest/docs/API_references/qua/cast/) peut être utile).
- Tracer les composantes I et Q du signal de lecture en fonction du délai.
- Estimer le temps $T_2$ du qubit.


In [ ]:
# Parameters Definition
n_avg = 2000  # The number of averages
# Pulse duration sweep (in clock cycles = 4ns) - must be larger than 4 clock cycles
t_min = 4 // 4
t_max = 200000 // 4
dt = 10000 // 4
durations = np.arange(t_min, t_max, dt)
thermalization_time = 80 #in µs

###################
# The QUA program #
###################
with program() as qmprog:
    n = declare(int)  # QUA variable for the averaging loop
    t = declare(int)  # QUA variable for the qubit pulse duration
    I = declare(fixed)  # QUA variable for the measured 'I' quadrature
    Q = declare(fixed)  # QUA variable for the measured 'Q' quadrature
    I_st = declare_stream()  # Stream for the 'I' quadrature
    Q_st = declare_stream()  # Stream for the 'Q' quadrature

    with for_(n, 0, n < n_avg, n + 1):  # QUA for_ loop for averaging
        with for_(*from_array(t, durations)):  # QUA for_ loop for sweeping the waiting time
            # Write the sequence
            # ....
            # Measure the state of the resonator
            # Do not forget to wait for the end of the pulse on the qubit to perfom the measurement
            # (Use the align command)
            measure(
                "readout",
                "resonator",
                dual_demod.full("cos", "sin", I),
                dual_demod.full("minus_sin", "cos", Q),
            )
            # Wait for the qubit to decay to the ground state
            wait(thermalization_time * u.us, "resonator")
            # Save the 'I' & 'Q' quadratures to their respective streams
            save(I, I_st)
            save(Q, Q_st)

    with stream_processing():
        # Cast the data into a 1D vector, average the 1D vectors together and store the results on the OPX processor
        I_st.buffer(len(durations)).average().save("I")
        Q_st.buffer(len(durations)).average().save("Q")

# Send the QUA program to the OPX, which compiles and executes it
job = QM.Job(qmprog)

# Non interacting version
# job = QM.JobSimple(qmprog)
# Wait for the job to finish
# job.wait()

In [ ]:
# Create plot
I,Q=job.get_results("I", "Q")
fig,ax = plt.subplots()
ax.plot(4*durations*1e-3, I*1e4, 4*durations*1e-3, Q*1e4)
ax.set_ylabel("Quadrature")
ax.set_xlabel("Pulse Delay [µs]")
ax.legend(('I','Q'))

## Exercice 4 : Spectroscopie de la transition $|1\rangle \rightarrow |2\rangle$

- À partir de la séquence de spectroscopie du qubit dans `00_Spectroscopie`, écrire une séquence permettant de réaliser la spectroscopie de l’état $|2\rangle$.
